In [638]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
# from torch.autograd import Variable
import random
import wandb
# from torch.nn.utils.rnn import masked_cross_entropy
# random.seed(10)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [639]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [640]:
# !unzip /content/gdrive/MyDrive/aksharantar_sampled.zip > /dev/null

In [641]:
USE_CUDA = False

In [642]:
OUTPUT_LANGUAGE = 'mni'
MAX_WORD_LENGTH = 25

## READING DATA

In [643]:
'''
Below train, validation and test data are list of strings
'''
train_data=pd.read_csv('aksharantar_sampled/'+OUTPUT_LANGUAGE+'/'+OUTPUT_LANGUAGE+'_train.csv',header=None)
X_train = list(train_data[0])
Y_train = list(train_data[1])

validation_data=pd.read_csv('aksharantar_sampled/'+OUTPUT_LANGUAGE+'/'+OUTPUT_LANGUAGE+'_valid.csv',header=None)
X_valid = list(validation_data[0])
Y_valid = list(validation_data[1])

test_data=pd.read_csv('aksharantar_sampled/'+OUTPUT_LANGUAGE+'/'+OUTPUT_LANGUAGE+'_test.csv',header=None)
X_test = list(test_data[0])
Y_test = list(test_data[1])

In [644]:
SOS_token = 0
EOS_token = 1
UNK_TOKEN = 2
# ND_TOKEN = 
class Lang:
    def __init__(self, name):
        self.name = name
        self.chr2index = {}
        self.chr2count = {}
        # self.index2chr = {0: "$", 1: "*", 2: "_"}
        self.index2chr = {0: "$", 1: "*"}
        self.n_chrs = 2  # 3 if Count SOS and EOS and UNK

    def addword(self, word):
        for letter in word:
            self.addchr(letter)

    def addchr(self, chr_):
        if chr_ not in self.chr2index:
            self.chr2index[chr_] = self.n_chrs
            self.chr2count[chr_] = 1
            self.index2chr[self.n_chrs] = chr_
            self.n_chrs += 1
        else:
            self.chr2count[chr_] += 1
            
    def add_word_list(self,word_lst):
        for word in word_lst:
            self.addword(word)

In [645]:
inp_lang = Lang("eng")
inp_lang.add_word_list(X_train)

out_lang = Lang(OUTPUT_LANGUAGE)
out_lang.add_word_list(Y_train)


In [646]:
print(inp_lang.chr2index)

{'m': 2, 'a': 3, 't': 4, 'h': 5, 'n': 6, 'g': 7, 'i': 8, 's': 9, 'l': 10, 'o': 11, 'u': 12, 'k': 13, 'r': 14, 'e': 15, 'p': 16, 'c': 17, 'j': 18, 'b': 19, 'd': 20, 'w': 21, 'y': 22, 'f': 23, 'v': 24, 'z': 25, 'x': 26, 'q': 27}


## DATA PROCESSING

In [647]:
# def get_one_hot(char_dict,ch,len_alphabets):
#   input_index = char_dict[ch]
#   one_hot_tensor = np.zeros(len_alphabets)
#   one_hot_tensor[input_index] = 1
#   return one_hot_tensor.tolist()

def get_one_hot(char_dict,ch,len_alphabets):
  # if ch not in char_dict.keys():
  #   return UNK_TOKEN
  # else:
  #   return char_dict[ch]
  
  return char_dict[ch]

# def data_processing(data,char_dict,len_chrs):
#   ONE_HOT = []
#   for word in data :
#     word = '$' + word + '*'
#     encoded_word = [get_one_hot(char_dict,i,len_chrs) for i in word]
#     encoded_word = torch.tensor(encoded_word)
#     ONE_HOT.append(encoded_word)
#   return ONE_HOT

# Hindi -> 28
# Tamil -> 30

# MAX_SEQ_LEN = 28

# def add_padding(word):
#   while len(word) != MAX_SEQ_LEN:
#     word += '_'
#   return word

def data_processing(data,char_dict,len_chrs):
  ONE_HOT = []
  for word in data :
    encoded_word = [get_one_hot(char_dict,i,len_chrs) for i in word]
    encoded_word.append(EOS_token)
    encoded_word = torch.LongTensor(encoded_word)
    ONE_HOT.append(encoded_word)
  return ONE_HOT

$ - Start Character \\
\* - End Character

###Hindi

In [648]:
# tgt_chrs = '$'
# for i in range(2304,2432):
#   tgt_chrs += chr(i)
# tgt_chrs += '*'
# tgt_char_dict = {char: i for i, char in enumerate(tgt_chrs)}

# inp_chrs = '$abcdefghijklmnopqrstuvwxyz*'
# inp_char_dict = {char: i for i, char in enumerate(inp_chrs)}

In [649]:
print(inp_lang.index2chr)
print(inp_lang.chr2index)

{0: '$', 1: '*', 2: 'm', 3: 'a', 4: 't', 5: 'h', 6: 'n', 7: 'g', 8: 'i', 9: 's', 10: 'l', 11: 'o', 12: 'u', 13: 'k', 14: 'r', 15: 'e', 16: 'p', 17: 'c', 18: 'j', 19: 'b', 20: 'd', 21: 'w', 22: 'y', 23: 'f', 24: 'v', 25: 'z', 26: 'x', 27: 'q'}
{'m': 2, 'a': 3, 't': 4, 'h': 5, 'n': 6, 'g': 7, 'i': 8, 's': 9, 'l': 10, 'o': 11, 'u': 12, 'k': 13, 'r': 14, 'e': 15, 'p': 16, 'c': 17, 'j': 18, 'b': 19, 'd': 20, 'w': 21, 'y': 22, 'f': 23, 'v': 24, 'z': 25, 'x': 26, 'q': 27}


In [650]:
# print(len(inp_lang.chr2index))
# print(len(out_lang.chr2index))
INP_LANG_SIZE = len(inp_lang.index2chr)
OUT_LANG_SIZE = len(out_lang.index2chr)
print(INP_LANG_SIZE)
print(OUT_LANG_SIZE)

28
46


Below is the template to create your dataset for the langauage you want to train on

In [651]:
'''
start -> unicode of the first character in the language
end -> unicode of the last character in the language

tgt_chrs = '$'
for i in range(start,end):
  tgt_chrs += chr(i)
tgt_chrs += '*_'
tgt_char_dict = {char: i for i, char in enumerate(tgt_chrs)}

inp_chrs = '$abcdefghijklmnopqrstuvwxyz*_'
inp_char_dict = {char: i for i, char in enumerate(inp_chrs)}
'''

"\nstart -> unicode of the first character in the language\nend -> unicode of the last character in the language\n\ntgt_chrs = '$'\nfor i in range(start,end):\n  tgt_chrs += chr(i)\ntgt_chrs += '*_'\ntgt_char_dict = {char: i for i, char in enumerate(tgt_chrs)}\n\ninp_chrs = '$abcdefghijklmnopqrstuvwxyz*_'\ninp_char_dict = {char: i for i, char in enumerate(inp_chrs)}\n"

In [652]:
# # These are lists of tensors corresponding to each word
inp_train = data_processing(X_train,inp_lang.chr2index,len(inp_lang.chr2index))
tgt_train = data_processing(Y_train,out_lang.chr2index,len(out_lang.chr2index))
inp_valid = data_processing(X_valid,inp_lang.chr2index,len(inp_lang.chr2index))
tgt_valid = data_processing(Y_valid,out_lang.chr2index,len(out_lang.chr2index))
inp_test = data_processing(X_test,inp_lang.chr2index,len(inp_lang.chr2index))
tgt_test = data_processing(Y_test,out_lang.chr2index,len(out_lang.chr2index))

In [653]:
# print(tgt_train[0].size())

In [654]:
# print(max([len(X_train[i]) for i in range(len(X_train))]))
# print(max([len(X_test[i]) for i in range(len(X_test))]))
# print(max([len(X_valid[i]) for i in range(len(X_valid))]))
# print(max([len(Y_train[i]) for i in range(len(Y_train))]))
# print(max([len(Y_test[i]) for i in range(len(Y_test))]))
# print(max([len(Y_valid[i]) for i in range(len(Y_valid))]))

## ENCODER

In [655]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_layer_size, num_encoder_layers, cell_type, dropout_prob, bidirectional):
      super(Encoder, self).__init__()
      '''
        self.input_size : int
        self.hidden_layer_size : int
        self.num_encoder_layers : int
        self.cell_type : string
      '''
      self.input_size = input_size
      self.hidden_layer_size = hidden_layer_size
      self.num_encoder_layers = num_encoder_layers
      self.cell_type = cell_type

      self.embedding = nn.Embedding(self.input_size, embedding_size)

      if cell_type == 'RNN':
        self.rnn = nn.RNN(input_size = embedding_size, hidden_size = hidden_layer_size, num_layers = num_encoder_layers, dropout = dropout_prob, bidirectional = bidirectional)
      elif cell_type == 'LSTM':
        self.rnn = nn.LSTM(input_size = embedding_size, hidden_size = hidden_layer_size, num_layers = num_encoder_layers, dropout = dropout_prob, bidirectional = bidirectional)
      elif cell_type == 'GRU':
        self.rnn = nn.GRU(input_size = embedding_size, hidden_size = hidden_layer_size, num_layers = num_encoder_layers, dropout = dropout_prob, bidirectional = bidirectional)
      self.dropout = nn.Dropout(dropout_prob)
      self.D = 1
      if bidirectional == True :
        self.D = 2

    def forward(self, input, prev_hidden):
      # print("entered_encoder")
      seq_len = len(input)
      embedded = self.embedding(input)
      embedded = embedded.view(seq_len,1,-1)
      # print(embedded.size())
      # seq_len,1,embedding_size
      embedded = self.dropout(embedded)
      if self.cell_type == 'RNN':
        output, hidden = self.rnn(embedded,prev_hidden)
      elif self.cell_type == 'LSTM':
        cell = torch.zeros(self.num_encoder_layers,self.batch_size,self.hidden_layer_size)
        output, (hidden,cell) = self.rnn(embedded,(prev_hidden,cell))
      elif self.cell_type == 'GRU':
        output, hidden = self.rnn(embedded,prev_hidden)
        
      return output,hidden

    def init_hidden(self):
      
      '''
      # cell = Variable(torch.zeros(self.num_encoder_layers,batch_size,self.hidden_layer_size))
      if self.cell_type == 'LSTM':
        # cell = Variable(torch.zeros(self.num_encoder_layers,batch_size,self.hidden_layer_size))
        hidden = Variable(torch.zeros(self.num_encoder_layers,batch_size,self.hidden_layer_size))
      else :
        hidden = Variable(torch.zeros(self.num_encoder_layers,batch_size,self.hidden_layer_size))
      ''' 
      
      if self.cell_type == 'LSTM':
        hidden = torch.zeros(self.num_encoder_layers,1,self.hidden_layer_size)
      else :
        hidden = torch.zeros(self.num_encoder_layers,1,self.hidden_layer_size)
      # if USE_CUDA: hidden = hidden.cuda()
      # return hidden,cell
      return hidden


## DECODER

In [656]:
# class Decoder

class Decoder(nn.Module):
  def __init__(self, output_size, embedding_size, hidden_layer_size, num_layers, cell_type, dropout_prob, bidirectional):
    super(Decoder, self).__init__()
    '''
      self.output_size : int
      self.hidden_layer_size : int
      self.num_encoder_layers : int
      self.cell_type : string
      self.rnn : RNN,LSTM,GRU 
    '''
    self.output_size = output_size
    self.hidden_layer_size = hidden_layer_size
    self.num_layers = num_layers
    self.cell_type = cell_type
    self.embedding_size = embedding_size
    self.embedding = nn.Embedding(OUT_LANG_SIZE, embedding_size)

    if cell_type == 'RNN':
      self.rnn = nn.RNN(input_size = embedding_size, hidden_size = hidden_layer_size, num_layers = num_layers, dropout = dropout_prob, bidirectional = bidirectional)
    elif cell_type == 'LSTM':
      self.rnn = nn.LSTM(input_size = embedding_size, hidden_size = hidden_layer_size, num_layers = num_layers, dropout = dropout_prob, bidirectional = bidirectional)
    elif cell_type == 'GRU':
      self.rnn = nn.GRU(input_size = embedding_size, hidden_size = hidden_layer_size, num_layers = num_layers, dropout = dropout_prob, bidirectional = bidirectional)

    self.dropout = nn.Dropout(dropout_prob)
    self.fc = nn.Linear(hidden_layer_size, output_size)
    self.softmax = nn.LogSoftmax(dim = 1)
      
  def forward(self, input, prev_hidden):
    # print("Entered Decoder")
    embedded = self.embedding(input)
    embedded = embedded.view(1,1,-1)
    embedded = F.relu(embedded)
    embedded = self.dropout(embedded)
    # print("emb size",embedded.size())
    # print("hidden size",prev_hidden.size())
    # print()
    # print(embedded.size())
    # print(prev_hidden.size())
    if self.cell_type == 'RNN':
      output, hidden = self.rnn(embedded,prev_hidden)
    elif self.cell_type == 'LSTM':
      cell = torch.zeros(self.num_encoder_layers,self.batch_size,self.hidden_layer_size)
      output,(cell, hidden) = self.rnn(embedded,prev_hidden)
    elif self.cell_type == 'GRU':
      output, hidden = self.rnn(embedded,prev_hidden)
    # print("hjk")
    y_pred = self.fc(output)
    
    return y_pred, hidden

  # def init_hidden(self,batch_size):
  #   # return torch.zeros(self.num_layers,self.hidden_layer_size)
  #   if self.cell_type == 'LSTM':
  #       return torch.zeros(self.num_encoder_layers,self.batch_size,self.hidden_layer_size)
  #   else :
  #     return torch.zeros(self.num_encoder_layers,self.batch_size,self.hidden_layer_size)


In [657]:
# fd = torch.zeros(5)
# print(len(fd))

## SEQ2SEQ

In [658]:

class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, learning_rate, clip = 5.0, teacher_forcing_ratio = 0.5):
    super(Seq2Seq, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.teacher_forcing_ratio = teacher_forcing_ratio
    self.encoder_optimizer = optim.Adam(self.encoder.parameters(), lr = learning_rate)
    self.decoder_optimizer = optim.Adam(self.decoder.parameters(), lr = learning_rate)
    self.encoder_optimizer.zero_grad()
    self.decoder_optimizer.zero_grad()
    self.criterion = nn.NLLLoss()
    self.clip = clip
    self.teacher_forcing_ratio = teacher_forcing_ratio
    # self.batch_size = batch_size
      
  def forward(self, input, target):
    inp_len = input.size()[0]
    tgt_len = target.size()[0]
    loss = 0
    '''
    input : seq_length_inp
    target : seq_length_tgt
    '''
    self.encoder_optimizer.zero_grad()
    self.decoder_optimizer.zero_grad()
    # Encoder
    encoder_hidden = self.encoder.init_hidden()
    # print("here 345")
    encoder_outputs,encoder_hidden = self.encoder(input,encoder_hidden)
    # print("done")
    # print("target",target.size())
    # encder_hidden,encoder_cell = self.encoder.init_hidden(batch_size)
    # if self.encoder.cell_type == 'LSTM':
    #   encoder_output,(encoder_hidden,encoder_cell) = self.encoder(input,encoder_hidden)
    # else:
    # encoder_output, encoder_hidden = self.encoder(input,encoder_hidden)
    # for i in range(seq_len):
    #   encoder_input = input[i]
    #   encoder_output,encoder_hidden = self.encoder(encoder_input,encoder_hidden) # encoder_hidden size is num_layers,hidden_size
    '''
    encoder_hidden : num_layers,batch_size,embedding_size
    encoder_output : max_input_length,batch_size,embedding_size
    '''
    # print("here")
    # print(encoder_hidden.size())
    # Decoder
    decoder_input = torch.LongTensor([[SOS_token]])
    if USE_CUDA:
        decoder_input = decoder_input.cuda()
    decoder_hidden = encoder_hidden

    # decoder_outputs = Variable(torch.zeros(max_len,batch_size,self.decoder.output_size))
    # if USE_CUDA:
    #     decoder_outputs = decoder_outputs.cuda()
    
    # print("decoder_inp",decoder_input.size())
    # # Without Teacher Forcing
    # for i in range(1,tgt_len):
    #   decoder_output,decoder_hidden = self.decoder(decoder_input,decoder_hidden)
    #   loss += self.criterion(decoder_output.view(-1),target[i])
    #   decoder_input = target[i]

    # With Teacher Forcing
    use_teacher_forcing = random.random() < self.teacher_forcing_ratio
    # use_teacher_forcing = False
    if use_teacher_forcing:
      for i in range(tgt_len):
        # print("just before entering decoder")
        decoder_output,decoder_hidden = self.decoder(decoder_input,decoder_hidden)
        # print(decoder_output.size())
        # print(target[i])
        loss += self.criterion(decoder_output.view(-1),target[i])
        # print("after calculating loss")
        # decoder_outputs[i] = decoder_output
        decoder_input = target[i]
        print()
    else :
      for i in range(tgt_len):
        # print(decoder_input.size())
        decoder_output,decoder_hidden = self.decoder(decoder_input,decoder_hidden)
        # print("deco",decoder_output.size())
        loss += self.criterion(decoder_output.view(-1),target[i])
        topv,topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        # print("ni ",ni)
        # decoder_outputs[i] = decoder_output
        # print(decoder_output.size())
        decoder_input = torch.LongTensor([[ni]])
        if ni == EOS_token : 
          break
    # print("done full decoder")
    loss.backward()
    torch.nn.utils.clip_grad_norm_(self.encoder.parameters(), self.clip)
    torch.nn.utils.clip_grad_norm_(self.decoder.parameters(), self.clip)
    self.encoder_optimizer.step()
    self.decoder_optimizer.step()
    return loss.data/tgt_len

  def predict(self,input):
    inp_len = input.size()[0]
    encoder_hidden = self.encoder.init_hidden()
    encoder_outputs,encoder_hidden = self.encoder(input,encoder_hidden)
    
    # Decoder
    decoder_input = torch.LongTensor([[SOS_token]])
    decoder_hidden = encoder_hidden

    decoded_words = []
    pred = ''
    for i in range(MAX_WORD_LENGTH):
      decoder_output,decoder_hidden = self.decoder(decoder_input,decoder_hidden)
      topv,topi = decoder_output.data.topk(1)
      ni = topi[0][0]
      decoder_input = torch.LongTensor([[ni]])
      decoded_words.append(ni)
      pred += out_lang.index2chr(ni)
      if ni == EOS_token : 
        break
    return [torch.LongTensor(decoded_words),pred]


In [659]:
tg = torch.LongTensor([[SOS_token]])
print(tg.size())

torch.Size([1, 1])


## Testing

In [660]:
# Testing

# clip = 4.0
# batch_size = 128

# encodertest = Encoder(30,32,10,5,'RNN',0.1,False,batch_size = batch_size)
# decodertest = Decoder(30,32,10,5,'RNN',0.1,False,batch_size = batch_size)
# Model = Seq2Seq(Encoder,Decoder)
# # print(encodertest)

# criterion_ = nn.CrossEntropyLoss()

# # input = inp_train[0]
# # target = tgt_train[0]

# # input_length = input.size()[0]
# # target_length = target.size()[0]

# encoder_hidden = encodertest.init_hidden()

# # Initial Implementation
# # encoder_outputs = ()
# # for i in range(0,len(inp_train),batch_size):
# #   curr_batch = min(len(inp_train)-i,batch_size)
# #   encoder_output, encoder_hidden = encodertest(input[i:i+curr_batch],encoder_hidden)
# #   print(encoder_output.size())
# #   encoder_outputs = encoder_outputs + (encoder_output,)
# # encoder_outputs = torch.stack(encoder_outputs)

# encoder_output, encoder_hidden = encodertest(inp_train[0:batch_size],encoder_hidden)
# encoder_hidden = encoder_hidden.permute(1,0,2)
# # print(encoder_hidden[encoder_hidden.size()[0]-1])


# decoder_input = tgt_train[0]
# decoder_hidden = encoder_hidden[encoder_hidden.size()[0]-1]
# loss = 0
# decoder_outputs = ()
# for i in range(1,len(tgt_train)):
#   decoder_output, decoder_hidden = decodertest(decoder_input, decoder_hidden)
#   loss += criterion_(decoder_output, tgt_train[i].to(torch.long))
#   decoder_input = tgt_train[i]
# loss.backward()
# torch.nn.utils.clip_grad_norm_(encodertest.parameters(), clip)
# torch.nn.utils.clip_grad_norm_(decodertest.parameters(), clip)
# print(loss)



# encoder_optimizer.step()
# decoder_optimizer.step()



## Training

In [661]:
l = np.array([1,4,3,4])
z = np.array([3,4,5,4])
print(sum(l==z))

2


In [662]:
# def train_once(model,batch_size):
#   # Trains once on the whole dataset
#   training_loss = 0
#   # for i in range(len(inp_train)):
#     # if i % 10 == 0:
#     #   print(i)
#   train_length = inp_train.size(0)
#   for i in range(0,train_length,batch_size):
#     curr_batch = min(train_length-i,batch_size)
#     # print("iter = ",i)
#     input_tensor = inp_train[i:i+curr_batch]
#     target_tensor = tgt_train[i:i+curr_batch]
#     loss = model(input_tensor,target_tensor)
#     training_loss += loss/(curr_batch*(MAX_SEQ_LEN+2))
#   return training_loss

def train_once(model,batch_size):
  # Trains once on the whole dataset
  training_loss = 0
  train_length = len(inp_train)
  for i in range(train_length):
    input_tensor = inp_train[i]
    target_tensor = tgt_train[i]
    loss = model(input_tensor,target_tensor)
    training_loss += loss
    # if i%200 == 0:
    print(i)
  return training_loss

def accuracy_score(pred,actual):
  return np.sum(pred == actual)/len(pred)

def eval_acc(model):
  train_pred = [model.predict(i) for i in inp_train]
  valid_pred = [model.predict(i) for i in inp_valid]
  train_pred = list(map(list, zip(*train_pred)))[1]
  valid_pred = list(map(list, zip(*valid_pred)))[1]
  train_accuracy = accuracy_score(train_pred,Y_train)
  valid_accuracy = accuracy_score(valid_pred,Y_valid)
  return train_accuracy,valid_accuracy

# def eval_loss(model):
  
  

# def get_accuracy(model,pred,target):
#     acc = sum(int([model.predict(input[i])[1] == target[i] for i in range(input.size(0))]))/input.size(0)
#     return acc

# def get_accuracies(model):
#   train_accuracy = get_accuracy(model,inp_train,tgt_train)
#   valid_accuracy = get_accuracy(model,inp_valid,tgt_valid)
#   test_accuracy = get_accuracy(model,inp_test,tgt_test)
#   return train_accuracy,valid_accuracy,test_accuracy
  
def train(model,epochs,batch_size):
  for epoch in range(epochs):
    train_loss = train_once(model,batch_size)
    print("Epoch : %d, Training Loss : %f" % (epoch+1,train_loss))
    # train_accuracy,valid_accuracy = eval_acc(model)

# def random_train(model,epochs):
#   indices = [*range(epochs)]
#   loss = 0
#   for i in range(epochs):
#     index = random.choice(indices)
#     indices.remove(index)
#     inp_word,tgt_word = inp_train[index],tgt_train[index]
#     loss += model(inp_word,tgt_word)
#   return loss




## Run

In [663]:
# define hyperparameters
input_size = inp_lang.n_chrs
embedding_size = 128
output_size = out_lang.n_chrs
hidden_layer_size = 64
num_layers = 4
cell_type = 'RNN'
dropout_prob = 0.1
learning_rate = 0.01
bidirectional = False
batch_size = 4096
epochs = 10
encoder = Encoder(input_size, embedding_size, hidden_layer_size, num_layers, cell_type, dropout_prob, bidirectional)
decoder = Decoder(output_size, embedding_size, hidden_layer_size, num_layers, cell_type, dropout_prob, bidirectional)
model = Seq2Seq(encoder,decoder,learning_rate)
train(model,epochs,batch_size)
# print(model.predict(inp_train[0]))
# tgt_pred = model.predict(inp_valid)
# acc = accuracy(tgt_pred,tgt_valid)
# print(acc)


0
1










2
3






4





5








6








7
8









9
10







11
12






13








14







15
16
17








18









19






20
21










22
23
24
25
26









27










28








29
30








31






32









33








34











35
36







37









38
39





40
41








42
43









44
45
46
47
48
49
50
51









52
53





54
55
56
57
58




59






60
61
62
63
64








65






66
67
68
69






70








71
72





73
74






75
76






77









78







79
80







81







82
83






84
85
86








87








88
89





90






91
92




93










94







95











96
97
98










99










100










101
102
103
104







105







106
107
108
109
110








111





112
113
114
115










116
117
118








119








120








121







122
123







124






125






126









127









128
129







130
131









132
133







134
135
136
137








138








13

KeyboardInterrupt: 

In [ ]:
# train(model,epochs,batch_size)
# train(model,epochs,batch_size)
# train(model,epochs,batch_size)
# train(model,epochs,batch_size)
# train(model,epochs,batch_size)
# train(model,epochs,batch_size)
# train(model,epochs,batch_size)

In [ ]:
# tgt_red = model.predict(inp_valid)
# # print(tgt_pred)
# for i in range(tgt_pred.size(0)):
#     for j in range(tgt_pred.size(1)):
#         print(tgt_pred[i][j],end = ' ')
#     print()
#     for j in range(tgt_pred.size(1)):
#         print(tgt_valid[i][j],end = ' ')
#     print()
#     print()
# acc = accuracy(tgt_pred,tgt_valid)
# print(acc)

In [ ]:
sweep_config = {
    # "name" : "assignment_sweeps",
    "method" : "bayes",
    "metric" :{
        "name" : "validation_accuracy",
        "goal" : "maximize"
    },
    "parameters" : {
        "embedding_size" : {
            "values" : [16,32,64,128,256]
        },
        "num_layers" : {
            "values" : [1,2,3,4]
        },
        "hidden_layer_size" : {
            "values" : [16,32,64,128,256]
        },
        "learning_rate" : {
            "values" : [0.001,0.0001]
        },
        "cell_type" : {
           "values" : ['RNN', 'LSTM', 'GRU'] 
        },
        "dropout" : {
            "values" : [0.1,0.2,0.3]
        },
    }
}



In [ ]:
sweep_id = wandb.sweep(sweep_config,project="CS6910_Assignment_2")
wandb.agent(sweep_id = sweep_id,function = train,count = 40)